# SQL in Python Assignment

In [3]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table

### Connect to the personal database you created yesterday.
Remember the way the credentials work:
```
user = 'postgres' # Replace with your user, if different
pwd = 'pAsSwOrD' # Replace with your password
host = 'localhost'
port = '5432'
db = 'postgres' # Replace with your database name, if different
```

In [11]:
user = 'postgres' # Replace with your user, if different
pwd = 'pAsSwOrD' # Replace with your password
host = 'localhost'
port = '5433'
db = 'postgres' # Replace with your database name, if different

conn_str = 'postgresql://{}:{}@{}:{}/{}'.format(
    user, pwd, host, port, db)


In [12]:
engine = create_engine(conn_str)

### Obtain a list of tables that are in the database.

In [7]:
engine.table_names()

['vehicles', 'houseprices', 'normal_sales']

### Obtain a list of all the fields that are in the vehicles table along with the data type of each.

In [9]:
meta = MetaData(engine)

In [10]:
vehicles = Table("vehicles", meta, autoload=True)

### Write a SELECT query to retrieve all records from the database and load them into a Pandas data frame.

In [13]:
sql = '''
SELECT * 
FROM vehicles
'''

vehicles = pd.read_sql_query(sql, conn_str)
vehicles.head()

,id,make,model,year,class,trans,drive,cyl,displ,fuel,hwy,cty
0,27550,AM General,DJ Po Vehicle 2WD,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,4.0,2.5,Regular,17,18
1,28426,AM General,DJ Po Vehicle 2WD,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,4.0,2.5,Regular,17,18
2,27549,AM General,FJ8c Post Office,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,6.0,4.2,Regular,13,13
3,28425,AM General,FJ8c Post Office,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,6.0,4.2,Regular,13,13
4,1032,AM General,Post Office DJ5 2WD,1985,Special Purpose Vehicle 2WD,Automatic 3-spd,Rear-Wheel Drive,4.0,2.5,Regular,17,16


### Create a table that contains all Audi vehicles with a city fuel efficiency greater than 18.

In [24]:
create = '''
CREATE TABLE IF NOT EXISTS audi_cty_mpg_18 AS
SELECT *
FROM vehicles
WHERE make = 'Audi'
    AND cty > 18
'''

engine.execute(create)

### Delete all records from the temp table you just created.

In [25]:
delete = 'DELETE FROM audi_cty_mpg_18'

engine.execute(delete)

### Insert records from the vehicles table back into the table you created where the make is Audi and the city fuel efficiency is greater than 20.

In [26]:
insert = '''
INSERT INTO audi_cty_mpg_18
SELECT * 
FROM vehicles
WHERE make = 'Audi'
    AND cty > 20
'''

engine.execute(insert)

### Update the drive field of the records in the table you created to change any "4-Wheel or All-Wheel Drive" values to "All-Wheel Drive."

In [27]:
update = '''
UPDATE audi_cty_mpg_18
SET drive = 'All-Wheel Drive'
WHERE drive = '4-Wheel or All-Wheel Drive'
'''
engine.execute(update)

### Write a for loop that iterates over the list of makes provided below, selects the top 10 vehicles with the highest city fuel efficiency for each make, and then concatenates all the results together into a single Pandas data frame.

In [31]:
vehicles = Table('vehicles', meta, autoload=True, autoload_with=engine)
fields = [(c.name, c.type) for c in vehicles.columns]
fields_df = pd.DataFrame(fields, columns=['Name', 'Type'])

In [34]:
makes = ['Ford', 'Chevrolet', 'Toyota', 'Honda']
results = []


for make in makes:
    sql = "SELECT * FROM vehicles WHERE make = '" + make + "'ORDER BY cty DESC LIMIT 10"
          
    result = engine.execute(sql)
    results.append(pd.DataFrame(result, columns=list(fields_df.Name)))
    
results_df = pd.concat(results)

In [36]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 9
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      40 non-null     int64  
 1   make    40 non-null     object 
 2   model   40 non-null     object 
 3   year    40 non-null     int64  
 4   class   40 non-null     object 
 5   trans   36 non-null     object 
 6   drive   40 non-null     object 
 7   cyl     21 non-null     float64
 8   displ   21 non-null     float64
 9   fuel    40 non-null     object 
 10  hwy     40 non-null     int64  
 11  cty     40 non-null     int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 4.1+ KB
